# Hard Hat detection

## Forming the directory structure and selecting model

### Efficient det D0 512x512

In [ ]:
PROJECT_NAME = "hard_hat_detection"
CUSTOM_MODEL_NAME = 'hard_hat_detector_efficientdet_d0_200k' 
PRETRAINED_MODEL_NAME = 'efficientdet_d0_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
import os

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME, 'images'),
    'TRAIN_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME, 'images', 'train'),
    'TEST_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME, 'images', 'test'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace', PROJECT_NAME,'models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'RAW_DATASET_PATH': os.path.join("raw_datasets", PROJECT_NAME),
    'TRAIN_RAW_DATASET_PATH': os.path.join("raw_datasets", PROJECT_NAME, 'train'),
    'TEST_RAW_DATASET_PATH': os.path.join("raw_datasets", PROJECT_NAME, 'test'),
    'MEDIA_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME, 'images', 'media'),
    'VIOLATION_LOGGING_PATH': os.path.join('Tensorflow', 'workspace', PROJECT_NAME, 'images', 'violation_logging')
    }

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace', PROJECT_NAME, 'models',CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

## Preprocessing images and annotations

In [ ]:
import os
import cv2
import imghdr

data_directory = paths["RAW_DATASET_PATH"]
valid_extensions = ["jpg", "jpeg", "png", "bmp"]

for image_class in os.listdir(data_directory):
    print(f"Processing {image_class}...")
    for image in os.listdir(os.path.join(data_directory, image_class)):

        img_path = os.path.join(data_directory, image_class, image)
        file_name, file_extension = os.path.splitext(img_path)

        if(file_extension== ".xml"):
            continue

        try:
            img = cv2.imread(img_path)
            extension = imghdr.what(img_path)
            img_size = os.path.getsize(img_path)

            if img_size < 10000:
                print(f"File too small. Removing {image} from {image_class}")
                os.remove(img_path)

            if extension not in valid_extensions:
                print(f"Invalid image type. Removing image {image} from {image_class}")
                os.remove(img_path)

        except:
            print(f"Error reading file {image}. Removing from {image_class}.")
            os.remove(img_path)



print("--------------------------------------------------------------------------------")
for image_class in os.listdir(data_directory):
    print(f"Remaining image in {image_class} are {len(os.listdir(os.path.join(data_directory, image_class)))}")

In [ ]:
import glob
import os
import tensorflow as tf

data_directory = "my_dataset"

img_paths = glob.glob(os.path.join(data_directory,'*/*.*')) # assuming you point to the directory containing the label folders.

for image_path in img_paths:

    file_name, file_extension = os.path.splitext(image_path)

    if(file_extension== ".xml"):
        continue

    try:
        img_bytes = tf.io.read_file(image_path)
        decoded_img = tf.io.decode_image(img_bytes)
    except tf.errors.InvalidArgumentError as e:
        print(f"Found bad path {image_path}...{e}. Removing...")
        os.remove(image_path)

print("\n\nScan Completed")

In [ ]:
def move_files(source_directory, target_directory): 

    print(f"Processing {source_directory}")

    import shutil
    import os

    if not target_directory.endswith('/'):
        target_directory += '/'

    total_moved = 0
    valid_extensions = ["jpg", "jpeg", "png", "bmp"]

    for path in os.listdir(source_directory):

        file_name, file_extension = os.path.splitext(path)
        
        if file_extension[1:] in valid_extensions:
            if(os.path.exists(os.path.join(source_directory, file_name + '.xml'))):

                shutil.move(os.path.join(source_directory, file_name + '.xml'), target_directory)
                shutil.move(os.path.join(source_directory, file_name + file_extension), target_directory)

                print(f"Moved {file_name} and its annotation.")
                total_moved += 1

            else:
                print(f"No annotation found for {file_name}. Discarding...")
        else:
            continue

    remaining_files = [item for item in os.listdir(source_directory) if os.path.isfile(os.path.join(source_directory, item))]


    print("\n" + "-" * 20)
    print(f"Successfully moved {total_moved} images and their annotations")
    print(f"Remaining files: {len(remaining_files)}")
    print("\n" + "-" * 20)

In [ ]:
move_files(paths["TRAIN_RAW_DATASET_PATH"], paths["TRAIN_PATH"])
move_files(paths["TEST_RAW_DATASET_PATH"], paths["TEST_PATH"])

In [ ]:
import os
import xml.etree.ElementTree as ET

def is_valid_bbox(bndbox, width, height):
    xmin = int(bndbox.find('xmin').text)
    xmax = int(bndbox.find('xmax').text)
    ymin = int(bndbox.find('ymin').text)
    ymax = int(bndbox.find('ymax').text)

    if xmin < 0 or ymin < 0 or xmax > width or ymax > height or xmin >= xmax or ymin >= ymax:
        return False
    
    return True

def remove_invalid_xml_files(xml_file, valid_classes, directory):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    objects = root.findall('object')
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    for obj in objects:
        name = obj.find('name')
        bndbox = obj.find('bndbox')

        if name is not None and name.text not in valid_classes:
            root.remove(obj)

        if not is_valid_bbox(bndbox, width, height):

            os.remove(xml_file)
            img_file = os.path.join(directory, root.find('filename').text)

            if os.path.exists(img_file):
                os.remove(img_file)
            else:
                print(f"No corresponding image found for xml - {xml_file}")
                return 2
            
            return 1

    if len(root.findall('object')) == 0:  # No valid objects left
        os.remove(xml_file)
        img_file = os.path.join(directory, root.find('filename').text)
        if os.path.exists(img_file):
            os.remove(img_file)
        else:
            print(f"No corresponding image found for xml - {xml_file}")
            return 2

        return 1

    tree.write(xml_file)
    return 0

def process_directory(directory, valid_classes):

    removed_files = 0
    images_to_be_removed = []

    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            xml_file = os.path.join(directory, filename)

            process_xml_code = remove_invalid_xml_files(xml_file, valid_classes, directory)

            if process_xml_code == 1:
                print(f"Removed {filename} (and its corresponding image) due to invalid data.")
                removed_files += 1
            elif process_xml_code == 2:
                print(f"No corresponding image found for xml - {filename}. Please remove the image")
                images_to_be_removed.append(filename)
            else:
                print(f"Processed file: {filename}")

    print(f"Successfully removed {removed_files} files...")
    
    if(len(images_to_be_removed)):
        print("Remove the following images. (No corresponding xml files exist. ERROR CODE: 2)")

        for i in images_to_be_removed:
            print(i)
    



In [ ]:
# import os
# import xml.etree.ElementTree as ET

# def remove_invalid_objects(xml_file, valid_classes):
#     tree = ET.parse(xml_file)
#     root = tree.getroot()

#     objects = root.findall('object')

#     for obj in objects:
#         name = obj.find('name')
#         if name is not None and name.text not in valid_classes:
#             root.remove(obj)
#             return 1

#     tree.write(xml_file)

# def process_directory(directory, valid_classes):
#     for filename in os.listdir(directory):
#         if filename.endswith('.xml'):
#             xml_file = os.path.join(directory, filename)
#             if(remove_invalid_objects(xml_file, valid_classes)):
#                 print(f"Invalid object type. Removed {filename}")
#             else:
#                 print(f"Processed file: {filename}")

In [ ]:
valid_classes = ["helmet", "head"]

process_directory(paths["TRAIN_PATH"], valid_classes)
process_directory(paths["TEST_PATH"], valid_classes)

## Making label map for classes

In [ ]:
labels = [{'name':'helmet', 'id':1}, {'name':'head', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Download pre-trained model

In [ ]:
import wget

if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -xvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -xvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

## Generate TFRecord

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

## Configure the training pipeline

Make sure to apply the linear learning rate scaling described here: https://chatgpt.com/share/69662d90-f81a-4cd2-b9ff-48a810cf2271

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
# Extract default values from the config
default_batch_size = pipeline_config.train_config.batch_size
default_learning_rate_base = pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base
default_warmup_learning_rate = pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate

# New batch size
new_batch_size = 2
max_checkpoints = 50

print(default_learning_rate_base)
print(default_batch_size)

# Calculate the new learning rates
learning_rate_per_sample = default_learning_rate_base / default_batch_size
new_learning_rate_base = learning_rate_per_sample * new_batch_size
warmup_ratio = default_warmup_learning_rate / default_learning_rate_base
new_warmup_learning_rate = new_learning_rate_base * warmup_ratio

# Update the pipeline configuration
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = new_batch_size
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

# Update learning rates
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = new_learning_rate_base
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = new_warmup_learning_rate

# Print updated learning rates for confirmation
print(f"New learning rate base: {new_learning_rate_base}")
print(f"New warmup learning rate: {new_warmup_learning_rate}")

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## Training the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=200000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
import pyperclip

pyperclip.copy(command)

command

## Evaluating the model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
import pyperclip

pyperclip.copy(command)

command

### Tensorboard

In [ ]:
#To be executed after cd into models/custom_model_name/eval (or train)

#tensorboard --logdir=.



## Detection

#### Load the model

In [ ]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf



PATH_TO_CFG = files["PIPELINE_CONFIG"]
PATH_TO_CKPT = paths["CHECKPOINT_PATH"]

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-201')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files["LABELMAP"],
                                                                    use_display_name=True)

print(category_index)

#### Detect from image

In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
        path: the file path to the image

    Returns:
        uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

image_path = os.path.join(paths['IMAGE_PATH'], 'test', '000055_jpg.rf.16c9f577c3e94109f34389b632241d6c.jpg')
print('Running inference for {}... '.format(image_path), end='')

image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False)

plt.figure()
plt.imshow(image_np_with_detections)
print('Done')
plt.show()

# sphinx_gallery_thumbnail_number = 2

#### Detect from a video

In [ ]:
import cv2
import numpy as np
import cv2
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime, timedelta
from screeninfo import get_monitors
matplotlib.use('TkAgg')

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from datetime import datetime, timedelta
from screeninfo import get_monitors

# Path to your video file
VIDEO_PATH = os.path.join(paths["MEDIA_PATH"], "hard_hat_test5.mp4")
print(VIDEO_PATH)

# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)

label_id_offset = 1

head_class_id = None
for key, value in category_index.items():
    if value['name'] == 'head':
        head_class_id = key - label_id_offset
        break

if head_class_id is None:
    print("Error: 'head' class not found in category_index.")
    exit()

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

last_violation_time = datetime.now() - timedelta(seconds=15)
FRAME_SKIP_INTERVAL = 2
frame_count = 0

# Get screen resolution
screen = get_monitors()[0]
screen_width, screen_height = screen.width, screen.height

# Get the video's original frame rate
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the video in MP4 format
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = os.path.join(paths["MEDIA_PATH"], 'output_video_without_hard_hat.mp4')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (screen_width, screen_height))

# Read and display video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Reached the end of the video.")
        break

    # frame_count += 1
    # if frame_count % FRAME_SKIP_INTERVAL != 0:
    #     continue

    # Resize frame to fit the screen
    frame_height, frame_width = frame.shape[:2]
    aspect_ratio = frame_width / frame_height
    new_width = screen_width
    new_height = int(screen_width / aspect_ratio)
    if new_height > screen_height:
        new_height = screen_height
        new_width = int(screen_height * aspect_ratio)
    resized_frame = cv2.resize(frame, (new_width, new_height))

    image_np = np.array(resized_frame)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5, #Lower value is used for faster processing, default is 200
            min_score_thresh=0.4,
            agnostic_mode=False)

    head_detected = False
    for i in range(num_detections):
        if (detections['detection_scores'][i] > 0.4 and detections['detection_classes'][i] == head_class_id):
            head_detected = True
            break

    # Save the image if 'head' class is detected and 5 seconds have passed since the last violation
    current_time = datetime.now()
    if head_detected and current_time - last_violation_time > timedelta(seconds=10):
        print(f"At {current_time} and last time {last_violation_time}")
        violation_image_path = os.path.join(paths["VIOLATION_LOGGING_PATH"], f"violation_{current_time.strftime('%Y-%m-%d_%H:%M:%S')}.jpg")
        cv2.imwrite(violation_image_path, image_np_with_detections)
        last_violation_time = current_time
        print(f"Violation detected. Image saved at: {violation_image_path}\n")

    # Display the frame
    cv2.imshow('Video', image_np_with_detections)

    # Write the frame to the output video file
    out.write(cv2.resize(image_np_with_detections, (screen_width, screen_height)))

    # Exit the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()


## Freezing the graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

## Converting to TFJS

In [ ]:
import tensorflowjs #Not yet available for windows

In [ ]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])


In [ ]:
print(command)

In [ ]:
!{command}

## Conversion to TFLite

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [ ]:
print(command)

In [ ]:
import object_detection

!{command}

In [ ]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [ ]:
print(command)

In [ ]:
!{command}